<a href="https://colab.research.google.com/github/Dhanasree-Rajamani/Data-Mining/blob/main/Data%20Mining%20Assignment%205/Nearest_neighbor_FAISS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nearest neighbor(Similarity search) : Using FAISS and ANNOY libraries

Methods:

- Exhaustive Search

- Product Quantization

- Trees and Graphs

- Hierarchical Navigable Small Worlds (HNSW)



Install and Import Statements

In [ ]:
!pip install nmslib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.5 MB 201 kB/s 
     |████████████████████████████████| 188 kB 18.5 MB/s 


In [ ]:
!pip install faiss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.7 MB/s 


In [ ]:
!apt-get install libomp-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 5 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 0s (1,058 kB/s)
Selecting previously unselected package libomp5:amd64.
(Reading database ... 123991 files and directories currently installed.)
Preparing to unpack .../libomp5_5.0.1-1_amd64.deb ...
Unpacking libomp5:amd64 (5.0.1-1) ...
S

In [ ]:
!pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 647 kB 5.1 MB/s 
  Created wheel for annoy: filename=annoy-1.17.1-cp37-cp37m-linux_x86_64.whl size=395184 sha256=1fc7751fd654c56e432d1ddd8430d956f7cb0aef88a0deb95528f19981165d51
  Stored in directory: /root/.cache/pip/wheels/81/94/bf/92cb0e4fef8770fe9c6df0ba588fca30ab7c306b6048ae8a54
Successfully built annoy


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits

Loading Dataset

In [ ]:
digits = load_digits(n_class = 9)
X = digits.data
y = digits.target
n_samples, n_features = X.shape
X.shape

(1617, 64)

In [ ]:
X

array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., 10.,  0.,  0.],
       [ 0.,  0.,  0., ..., 16.,  9.,  0.],
       ...,
       [ 0.,  0.,  6., ...,  6.,  0.,  0.],
       [ 0.,  0.,  1., ...,  6.,  0.,  0.],
       [ 0.,  0., 10., ..., 12.,  1.,  0.]])

In [ ]:
X[0]

array([ 0.,  0.,  5., 13.,  9.,  1.,  0.,  0.,  0.,  0., 13., 15., 10.,
       15.,  5.,  0.,  0.,  3., 15.,  2.,  0., 11.,  8.,  0.,  0.,  4.,
       12.,  0.,  0.,  8.,  8.,  0.,  0.,  5.,  8.,  0.,  0.,  9.,  8.,
        0.,  0.,  4., 11.,  0.,  1., 12.,  7.,  0.,  0.,  2., 14.,  5.,
       10., 12.,  0.,  0.,  0.,  0.,  6., 13., 10.,  0.,  0.,  0.])

Transforming the dataset

In [ ]:
from sklearn.manifold import TSNE

tsne_2 = TSNE()
data_embeddings_tsne_2 = tsne_2.fit_transform(X)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


In [ ]:
data_embeddings_tsne_2.shape

(1617, 2)

In [ ]:
data_embeddings_tsne_2

array([[-62.524895  ,  -2.25274   ],
       [ -6.906851  ,  11.432599  ],
       [  7.1306977 ,  13.209991  ],
       ...,
       [-55.61915   ,   4.071234  ],
       [  8.460203  ,   2.5724685 ],
       [ 16.191196  ,   0.79502225]], dtype=float32)

Exhaustive Search

In [ ]:
import faiss

In [ ]:
exhaustive_search = faiss.IndexFlatL2(data_embeddings_tsne_2.shape[1])
exhaustive_search.add(data_embeddings_tsne_2)

In [ ]:
distances, indices = exhaustive_search.search(data_embeddings_tsne_2[80:81], 10) 
exhaustive_search_output_list = [y[i] for i in indices[0]]

In [ ]:
exhaustive_search_output_list

[6, 6, 6, 6, 6, 6, 6, 6, 6, 6]

Product Quantization

In [ ]:
number_of_partition = 8
search_in_x_partitions = 2
subvector_size = 8

pq_quantizer = faiss.IndexFlatL2(data_embeddings_tsne_2.shape[1])
pq = faiss.IndexIVFPQ(pq_quantizer, data_embeddings_tsne_2.shape[1], number_of_partition, search_in_x_partitions, subvector_size)
pq.train(data_embeddings_tsne_2)
pq.add(data_embeddings_tsne_2)

In [ ]:
distances, indices = pq.search(data_embeddings_tsne_2[90:91], 10) 
pq_output_list =  [y[i] for i in indices[0]]
pq_output_list

[1, 1, 1, 1, 1, 1, 1, 1, 1, 8]

Trees and Graphs

In [ ]:
import annoy

In [ ]:
number_of_trees = 5
annoy_index = annoy.AnnoyIndex(data_embeddings_tsne_2.shape[1])
for i, vec in enumerate(data_embeddings_tsne_2):
  annoy_index.add_item(i, vec.tolist())
annoy_index.build(number_of_trees)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  


True

In [ ]:
annoy_indices = annoy_index.get_nns_by_vector(data_embeddings_tsne_2[90].tolist(), 10)
annoy_output_list = [y[i] for i in annoy_indices]

In [ ]:
annoy_output_list

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [ ]:
graph_index = faiss.IndexHNSWFlat(data_embeddings_tsne_2.shape[1], 10)
graph_index.add(data_embeddings_tsne_2)

distances, indices = graph_index.search(data_embeddings_tsne_2[90:91], k=10)
graph_output_list = [y[i] for i in indices[0]]
graph_output_list

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Hierarchical Navigable Small Worlds (HNSW)

In [ ]:
import nmslib

In [ ]:
hnsw_index = nmslib.init(method = 'hnsw', space = 'cosinesimil')
hnsw_index.addDataPointBatch(data_embeddings_tsne_2)
hnsw_index.createIndex({'post': 2})

In [ ]:
y[80]

6

In [ ]:
hnsw_output = hnsw_index.knnQuery(data_embeddings_tsne_2[80], k = 10)
hnsw_output_list = [y[i] for i in hnsw_output[0]]
hnsw_output_list

[6, 6, 6, 6, 6, 6, 6, 6, 6, 6]